# Train Model
#### tensorflow_p36 environment

ref:  https://github.com/aws-samples/amazon-sagemaker-script-mode/blob/master/tf-eager-script-mode/tf-eager-sm-scriptmode.ipynb

Note:  AWS tutorials tend to name the post-training data = 'test'.   Most books call this 'val' for validation or 'eval' for model evaluation.   I named it 'val'.   So if you follow the example, AWS calls it 'test', I call it 'val'

## Step 3 - SageMaker HOSTED Training
At this point, you know you have a working training script (train.py).  So, you can have SageMaker deploy it to outside (not local) resources.  

### Output
After training in the HOSTED SageMaker environment, the model is pushed to S3.  This notebook pulls that newly trained model checkpoint to this (SageMaker) computer.   This notebook will then convert that checkpoint to a tflite model. 

In [ ]:
import os
import time
import sagemaker
from sagemaker.tensorflow import TensorFlow

In [ ]:
PROJECT_DIR = os.getcwd()

## Globals
It is helpful to increment the version number, you'll see this in the model artifacts

In [ ]:
PROJECT_DIR = os.getcwd()
TRAINED_MODEL_DIR = os.path.join(PROJECT_DIR, 'trained_model')

NUM_TRAINING_STEPS = 5000
VERSION = 104
PIPELINE_CONFIG = 'sagemaker_mobilenet_v1_ssd_retrain.config'
S3_PREFIX = 'cfa-datasciencesb-sagemaker'

## Data
SageMaker will pull the data from S3.    This is much faster than putting it in your Docker.   However, this is somewhat confusing because the MobileNet software (and utilities) were looking for data path in the config file.  We need to merge this approach:
- allow SageMaker to pull from S3
- AND, we want to continue leveraging the config design pattern

The other challenge is working with tarballs versus tfrecord files.

In [ ]:
traindata_s3_prefix = '{}/datasets/cfa_products/train'.format(S3_PREFIX)
valdata_s3_prefix = '{}/datasets/cfa_products/val'.format(S3_PREFIX)
print (traindata_s3_prefix)
print (valdata_s3_prefix)

## TIP
you would be wise to test and make sure you path is good before continuing!!  
cut/paste the printed value and put it into the following form.   You can run this AWS CLI command in a new cell.  

! aws s3 ls s3://cfaanalyticsresearch-sagemaker/datasets/cfa_products/train/  
! aws s3 ls s3://cfaanalyticsresearch-sagemaker/datasets/cfa_products/val/

### Copy data from local (SageMaker instance) to S3
If you ran the TrainModel_Step1 notebook, the data was moved to:
- code/tfrecords/train 
- code/tfrecords/val

In [ ]:
! pwd
# train_s3 == a full s3 URL, note that it is a folder, not a file
# this operation may take a few seconds (depending on data size) - it is silently copying
#     data from local drive on SageMaker to s3
train_s3 = sagemaker.Session().upload_data(path='./code/tfrecords/train/', key_prefix=traindata_s3_prefix)
val_s3 = sagemaker.Session().upload_data(path='./code/tfrecords/val/', key_prefix=valdata_s3_prefix)

inputs = {'train':train_s3, 'val': val_s3}

print(inputs)

In [ ]:
model_dir = '/opt/ml/model'     # this is related to how it gets deployed in the Docker
                                # this is a SAGEMAKER thing - don't confuse with the model_dir 
                                # that we have inside our code
# p2.xlarge == $1/hr
# p3.2xlarge = $3/hr
# this is a very controlled train & quick so the better server makes sense
# if you are developing - use the p2
train_instance_type = 'ml.p3.2xlarge'   
# TODO
#  o  try a different config for p3.2xlarge that has more images in the batch size to 
#     take advantage of the GPU memory
#  o  still have to figure out the data
#     - including the data under code/ directory means it is in the tarball
#     - but that means the inputs is a wasted step (and it takes longer to create the Docker image)
hyperparameters = {'pipeline_config_path' : 'sagemaker_mobilenet_v1_ssd_retrain.config',
                   'num_train_steps' : NUM_TRAINING_STEPS,
                   'num_eval_steps' : '1000'
                  }

# SageMaker Execution Role
role = sagemaker.get_execution_role()

### Tensorflow Version
aka here it is frame_version -- be sure to bump it up as you upgrade

In [ ]:
estimator = TensorFlow(entry_point='train115.py',
                       source_dir='code',
                       model_dir=model_dir,
                       train_instance_type=train_instance_type,
                       train_instance_count=1,
                       hyperparameters=hyperparameters,
                       role=role,
                       base_job_name='cfa-products-mobilenet-v1-ssd',
                       framework_version='1.14',
                       py_version='py3',
                       script_mode=True)

### Monitoring the Progress
You can watch the utilization of resources on the console.
- Training Jobs
- expand the active job

you'll see GPU utilization, memory, etc

In [ ]:
start_time = time.time()
print ("start time: {:.4f}".format(start_time))

# this will create the training job
# you can also see this on the SageMaker Training Job console
# 3 minute overhead prepping the servers, downloading data
# 5000 steps on a p3.2xlarge == 18 min (training time)
# overall ~ 20 minutes
estimator.fit(inputs)

# show the time
finish_time = time.time()
minutes = (finish_time - start_time) / 60
print("time spent: {:.4f}".format(finish_time - start_time))
print("in minutes: {:.4f}".format(minutes))

## SageMaker Model

https://sagemaker.readthedocs.io/en/stable/estimators.html

if you job completed successfully, you should see a message (above).   Something like:
Uploading generated training model
Training job completed.

If you don't export your model correctly - using the Exporter and the correct enironment variable values, you might not have anything to upload.   Your training script has to generate the correct artifacts for this proccess to work.

You can see output on S3 in the sagemaker bucket.  /output/model.tar.gz    In this tarball, you'll see /export/Servo/<version numer - which is a timestamp>/saved_graph.pb & variables
    
At this point you have a completed training job.   You do NOT have a SageMaker model.  i.e. look on the console:  Models - it's not there.   You have to create it next.

### Using the Console to go from Training Job -> Model
I'm sure there is a way to do this - but I don't understand it yet.
    
### Training Job Artifacts
#### Training Image  
You need the INFERENCE image
 
#### Output - S3 model artifact
this is what you need below
  
### Create Model
Models / Create  
name (example): model-mobilenet-ssdv1-cfa-products  
IAM Role:  you can use the SageMaker execution role  
(select) Profide model artificats and inference image location 
- location of inference code:  !!! you need this - I don't know what it is yet !!!
- location of model artifacts: s3://sagemaker-us-east-1-586454201570.s3.amazonaws.com/cfa-products-mobilenet-v1-ssd-2019-09-24-19-47-57-172/output/model.tar.gz
- container host name: blank

No VPC

## Conclusion
So, you can't complete this step.   The easiest workaround is, deploy the model using the code below.  When you deploy, it's automating all of this stuff anyway.

## [Optional] Retreiving the Trained Model
SageMaker created a Docker job to train our model and sent it off to external resources (external meaning - not this computer.)   Now we need to get the result - it's not on this computer.

- ./trained_model:  this local directory is (should be) empty
- the trained model is on s3 - in the next step we are copying the result to code/model
- now you'll see the tarball

/trained_model is NOT under the code/ directory.  Primarily because there is no reason to include it inside the Docker training job (in the event you re-run.)  That would just carry extra baggage around for no reason.

In [ ]:
print (estimator.output_path)
print (estimator.base_job_name)
print (estimator.model_uri)
print (estimator.checkpoint_s3_uri)
print (estimator.tags)
print (estimator.model_data)

In [ ]:
# current directory is still the top project directory - NOT the code directory
# this will get the current training artifacts
# !aws s3 cp {estimator.model_data} ./trained_model/model.tar.gz

# this will get the artifacts from a prevous job
model_data = estimator.model_data
print ("retrieve trained model:", model_data)
print ("copy to:", TRAINED_MODEL_DIR)
! aws s3 cp {model_data} {TRAINED_MODEL_DIR}
    

### model.ckpt-XXXX
Make note of the checkpoint files.   For example, if you  said run 5000 steps, there should be a checkpoint file:  
model.ckpt-5000*

This is the file that will be converted 
- frozen graph
- tflite model

#### CONFUSION WARNING - there is a difference between training locally and training with SageMaker
- Local, the checkpoint file is in code/model
- SageMaker job, you are about to extract the output to /trained_model  

The conversion script below is expecting the checkpoint to be in /trained_model

In [ ]:
!tar -xvzf ./trained_model/model.tar.gz -C ./trained_model

## Convert (Trained) Model Checkpoint to a tflite Model

WARNING: labels.txt - not included, don't think we need this though it was in the Coral project.  We think it's getting the label name from the label map (*.pbtxt) file

ALSO NOTE:  the script is named convert_checkpoint_to_edge_tflite.sh  
Well... the name is no longer totally accurate
- I took this from the original Coral TPU tutorial
- Another step is required for compiling for the EdgeTPU (not really relevant here since we are confined to AWS where there is no TPU -- so we skip that stuff)
- And, I added a step that converts the checkpoint to a TENSORFLOW frozen graph 
  - note that this generates a frozen_inference_graph.pb
  - it ALSO generates a saved model graph.pb  
  THESE (frozen graph & saved model) are NOT the SAME!!  
  https://stackoverflow.com/questions/46547319/error-when-parsing-graph-def-from-string

https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/exporting_models.md

In [ ]:
# WAKE UP - make sure the checkpoint num == hyperparameters/num_train_steps
# convert checkpoint is a task script - located in the tasks/ directory
os.chdir("tasks")  
! ./convert_checkpoint_to_edgetpu_tflite.sh --checkpoint_num {NUM_TRAINING_STEPS} --pipeline_config {PIPELINE_CONFIG}


In [ ]:
# set directory back to the project directory
# see the tflite model artifacts
os.chdir(PROJECT_DIR)
! ls tflite_model -l
! ls tensorflow_model -l

## Add the converted model artifacts to S3 (SageMaker Training Job Data)
Add these artifacts:
- frozen model graph
- tflite model

To the SageMaker folder that has all of the training job artifacts - so everything is together

In [ ]:
# estimator.model_data == the s3 url & file for the model output tarball
# we need the s3 url only; so you have to extract it; and, you need a / at the end
s3_model_artifacts = os.path.dirname(estimator.model_data) + '/'

# now you can copy these converted files up to s3
!aws s3 cp  ./tensorflow_model {s3_model_artifacts}tensorflow_model --recursive
!aws s3 cp  ./tflite_model {s3_model_artifacts}tflite --recursive

# Deploy

If you use this deploy method, it will build the model, endpoint config and the endpoint for you -- and it works.  I have had problems doing this manually.    So, until you figure out how to do it reliably as sub-steps.  It's easiest to just deploy, then delete the active endpoint.  

If you do that, then you can reliably recreate an endpoint from the endpoint config.

## Optional
This is not required.  You can take your model artifacts:
- tensorflow frozen graph
- tensorflow Lite frozen graph
- tensorflow Lite model file
They are completely useable.  Note that TensorFlow is quite different than TensorFlow Lite (which is much different than the EdgeTPU model.)  Missing is a TensorRT variant by the way.   You can test these models locally or on a different machine.   You don't have to deploy (but if you don't you'll lose the estimator object)

At this moment, you have an SageMaker Estimator.  This is a bunch of information about a model.   At this time, you can't recreate an Estimator from a file (i.e. restore/create from file).   So if you're going to deploy it - now is the time!   

### Version
see version in the GLOBALS  
if you use the same version number, it seems to reuse some stuff

## $$
When you deploy, you are paying for the endpoint server!! E.g. deploy your model to a p2.xlarge (which costs $1.26/hr) and you are paying whether you use it or not!  

### DON'T LEAVE YOUR ENDPOINT RUNNING

In [ ]:
# this is where you'll be glad you used a new version number

model_name = 'model-mobilenet-v1-ssd-cfa-products-{}'.format(VERSION)
endpoint_name = 'ep-mobilenet-v1-ssd-cfa-products-{}'.format(VERSION)
print ("model:", model_name)
print ("endpoint:", endpoint_name)

## SageMaker Model
Again, if you don't change the version number, it might re-use some stuff which makes it tougher to understand this step.

As the estimator.deploy() is in progress, you'll see:
- a model listed on the console
- an endpoint configuration
- an endpoint

you can now use that endpoint - but you can also re-use the endpoint configuration.

DON'T leave your endpoints running - this costs money - EC2 instance rates!

In [ ]:
# note, 
#  - I'm using a p2.2xlarge for SageMaker server
#  - We trained on a p3.2xlarge (sent the training job)
#  - now we can deploy to yet a 3rd machine - in this case, I'm selecting a p2 because it's the cheapest GPU

# THIS WILL TAKE A FEW MINUTES - Dockerizing (and it's probably carrying your data around if you put it in the code directory)
predictor = estimator.deploy(initial_instance_count=1,
                             instance_type='ml.p2.xlarge',
                             model_name=model_name,
                             endpoint_name=endpoint_name
                            )

Go to the console:
- Endpoints - you'll see ep-mobilenet-v1-ssd-cfa-products-<version> 
- Endpoint Configurations - you'll see it here
- Models - you'll see it here
    
S3 - you'll see the training job artifacts here

## Testing the Endpoint
Jump to the notebook:  DetectModel_Step2


### Delete the Endpoint

In [ ]:
sagemaker.Session().delete_endpoint(predictor.endpoint)